In [ ]:
import datetime # импорт datetime

class Account: # создание класса акаунт
    def __init__(self, account_holder: str, balance: float = 0):
        if balance < 0: # проверка стартового баланса 
            raise ValueError("Начальный баланс не может быть отрицательным.")
        self.holder = account_holder
        self._balance = balance
        self.operations_history = []  # Список для хранения операций

    def deposit(self, amount): # пополнение счета
        if amount <= 0:
            raise ValueError("Сумма пополнения должна быть положительной.")
        self._balance += amount
        self.addoperation('deposit', amount, 'success')

    def withdraw(self, amount):  #снятие средств
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной.")
        if self._balance >= amount:
            self._balance -= amount
            self.addoperation('withdraw', amount, 'success')
        else:
            self.addoperation('withdraw', amount, 'fail')

    def get_balance(self):
        return self._balance

    def get_history(self):
        return self.operations_history

    def addoperation(self, operationtype, amount, status):
        operation = {
            'type': operationtype,
            'amount': amount,
            'date_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),  # формат времени операции для журнала
            'balanceafter': self._balance,
            'status': status
        }
        self.operations_history.append(operation) #сохранение операций

class CreditAccount(Account):
    def __init__(self, account_holder: str, balance: float = 0, credit_limit: float = 0):
        super().__init__(account_holder, balance)
        self.credit_limit = credit_limit
        if credit_limit < 0:
            raise ValueError("Кредитный лимит не может быть отрицательным.")

    def withdraw(self, amount: float): # создадим функцию снятия средств
        if amount <= 0:
            raise ValueError("Сумма снятия должна быть положительной.")

        status = 'fail'
        credit_used = False

        #сравнение суммы расхода и кредитного лимита.
        if self._balance - amount >= -self.credit_limit:
            initial_balance = self._balance
            self._balance -= amount
            status = 'success'
            if initial_balance > 0 and self._balance < 0:
                credit_used = True
            elif initial_balance <= 0 and self._balance < initial_balance: # Если баланс кредитки меньше нуля то используем кредитные средства, ставим true
                credit_used = True
            elif initial_balance > 0 and self._balance >= 0 and self._balance < initial_balance: # Если баланс положительный то он им остается и кредит все еще доступен
                pass

        operation = {
            'type': 'withdraw',
            'amount': amount, # укажем количество снятия для журнала
            'date_time': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), # Укажем время операции для журнала
            'current_balance': self._balance if status == 'success' else self._balance, # Сделаем так что бы баланс не менялся если не удалось провести операцию
            'status': status,
            'credit_used': credit_used if status == 'success' else False
        }
        self.operations_history.append(operation)

    def get_available_credit(self) -> float: # функция выдачи в пределах кредитного лимита
        return self.credit_limit + self._balance
        
deb = Account("Елена", 7000)
deb.deposit(3500)
deb.withdraw(1050)
deb.withdraw(7000) 
deb.withdraw(1000) 
deb.withdraw(2000) 
print("Баланс:", deb.get_balance())
for row in deb.get_history():
    print(row)

print("Кредит")
cred = CreditAccount("Елена", balance=910, credit_limit=2800)
cred.withdraw(2450)   
cred.withdraw(14000)   
cred.withdraw(5000)
cred.deposit(560)    
cred.deposit(400)
print("Баланс:", cred.get_balance())
print("Доступные кредитные средства:", cred.get_available_credit())
for row in cred.get_history():
    print(row)


Баланс: 1450
{'type': 'deposit', 'amount': 3500, 'date_time': '2025-12-26 19:40:14', 'balanceafter': 10500, 'status': 'success'}
{'type': 'withdraw', 'amount': 1050, 'date_time': '2025-12-26 19:40:14', 'balanceafter': 9450, 'status': 'success'}
{'type': 'withdraw', 'amount': 7000, 'date_time': '2025-12-26 19:40:14', 'balanceafter': 2450, 'status': 'success'}
{'type': 'withdraw', 'amount': 1000, 'date_time': '2025-12-26 19:40:14', 'balanceafter': 1450, 'status': 'success'}
{'type': 'withdraw', 'amount': 2000, 'date_time': '2025-12-26 19:40:14', 'balanceafter': 1450, 'status': 'fail'}
Кредит
Баланс: -580
Доступные кредитные средства: 2220
{'type': 'withdraw', 'amount': 2450, 'date_time': '2025-12-26 19:40:14', 'current_balance': -1540, 'status': 'success', 'credit_used': True}
{'type': 'withdraw', 'amount': 14000, 'date_time': '2025-12-26 19:40:14', 'current_balance': -1540, 'status': 'fail', 'credit_used': False}
{'type': 'withdraw', 'amount': 5000, 'date_time': '2025-12-26 19:40:14', '